# QRT trials

## Exercise 1: Max Profit

Assume a trading strategy where we are given a list of events picked among the following list
- BUY stock units
- SELL stock units
- CHANGE stock delta_price
- QUERY

I start with 0 stocks, then I can buy stocks at price p, or sell them.
It is assumed that I dont short-sell (I sell only stocks that I have bought before, and maximum the total amount I own).
A CHANGE implies that the stock increases or decreases by delta_price. Thus adapting the profit.
A Query corresponds to the current profit given losses or profits of past BUY, SELL, CHANGE operations.

The function should take as input a list of events of type List[str] where each str corresponds to something like "BUY apple 20", or "CHANGE google -5", etc.

Here's an example

| Operation       | Portfolio           | Profit |
|-----------------|---------------------|--------|
| BUY apple 20    | apple 20            | 0      |
| BUY google 50   | apple 20, google 50 | 0      |
| QUERY           |                     | 0      |
| SELL apple 5    | apple 15, google 50 | 0      |
| CHANGE google 5 | apple 15, google 50 | 250    |
| CHANGE apple -2 | apple 15, google 50 | 220    |
| QUERY           |                     | 220    |

In [1]:
def getNetProfit(events):
    stock_values = {}  # {stock:value}
    profit = 0
    out = []

    for event in events:
        keywords = event.split(" ")
        action = keywords[0]
        if action == "BUY":
            stock, amount = keywords[1], float(keywords[2])
            if stock in stock_values:
                stock_values[stock] += amount
            else:
                stock_values[stock] = amount
        if action == "SELL":
            stock, amount = keywords[1], float(keywords[2])
            stock_values[stock] -= amount  # that's assuming shorting is not possible
        if action == "CHANGE":
            stock, delta_price = keywords[1], float(keywords[2])
            profit += int(delta_price * stock_values[stock])
        if action == "QUERY":
            out += [profit]
        print(keywords, profit, stock_values)

    return out


events = [
    "BUY stock2 2",
    "BUY stock1 4",
    "CHANGE stock2 -8",
    "SELL stock1 2",
    "BUY stock3 3",
    "QUERY",
]
getNetProfit(events)

['BUY', 'stock2', '2'] 0 {'stock2': 2.0}
['BUY', 'stock1', '4'] 0 {'stock2': 2.0, 'stock1': 4.0}
['CHANGE', 'stock2', '-8'] -16 {'stock2': 2.0, 'stock1': 4.0}
['SELL', 'stock1', '2'] -16 {'stock2': 2.0, 'stock1': 2.0}
['BUY', 'stock3', '3'] -16 {'stock2': 2.0, 'stock1': 2.0, 'stock3': 3.0}
['QUERY'] -16 {'stock2': 2.0, 'stock1': 2.0, 'stock3': 3.0}


[-16]

## Exercise 2: Possible sales

I have products identified by an id.
For each product per month, I can receive goods and sell goods.
I want to know if the schema is possible or not

Given a pandas dataframe containing the following columns:
- id: id of the product
- date_added : date of the reception or sale
- sizes: string containing the sizes of the elements sold or received separated by "/". It is assumed quantity is 1
- type: whether it is "Received" or "Sold"

Example:

| id | date_added | type     | sizes |
|----|------------|----------|-------|
| 1  | 2016-02    | Received | 7/8/9 |
| 1  | 2016-02    | Sold     | 7/8   |
| 2  | 2016-03    | Received | 36/37 |

In this example for ID 1, during the same month "2016-02", I receive 7/8/9 and sell 7/8. since 7 and 8 are part of what I received that particular ID seems possible.

The function should tell me if the provided df shows a "POSSIBLE" or "NOT POSSIBLE" df.

In [2]:
import pandas as pd

df = pd.DataFrame(
    {
        "id": [1, 1, 2],
        "date_added": ["2016-02", "2016-02", "2016-03"],
        "type": ["Received", "Sold", "Received"],
        "sizes": ["7/8/9", "8/9", "36/37"],
    }
)
df

,id,date_added,type,sizes
0,1,2016-02,Received,7/8/9
1,1,2016-02,Sold,8/9
2,2,2016-03,Received,36/37


In [3]:
from functools import reduce


def list_to_dict(l: list[int], count: int = 1) -> dict[int, int]:
    """
    Converts a list l to a dictionnary where list elements
    are keys and value is "count"
    """
    d = {}
    for e in l:
        d[e] = count
    return d


def inverse_dict_count(d: dict[int, int]) -> dict[int, int]:
    """
    Return {key: -values}
    """
    for key, value in d.items():
        d[key] = -value
    return d


def merge_dicts(d1, d2):
    """
    merges both dictionnaries with the following logic:
    - if a key is in both dictionnary, the output value is the sum of both
    - if a key is only in one dictionnary, the output value is the correspnding value
    """
    merged_dict = {}

    for key in set(d1) | set(d2):
        merged_dict[key] = d1.get(key, 0) + d2.get(key, 0)

    return merged_dict


def custom_agg(series):
    return reduce(merge_dicts, series)


def validate_date(d):
    """
    If a group has a negative value it means I sold more than I received
    As a consequence the answer to the question "is this possible?" is False
    for this particular id/date combo;
    """
    for key, value in d.items():
        if value < 0:
            return False
    return True


def main_function(df):
    # Convert strings with "/" to lists then dictionaries with values 1
    df["sizes_list"] = df.sizes.str.split("/").map(lambda x: list_to_dict(x, 1))
    print(df, "\n")
    # Convert values to -1 for sales
    df.loc[df["type"] == "Sold", "sizes_list"] = df.loc[
        df["type"] == "Sold", "sizes_list"
    ].map(inverse_dict_count)
    print(df, "\n")
    # Aggregate by item id and date by merging dictionaries
    df = df.groupby(by=["id", "date_added"], as_index=False).agg(
        {"sizes_list": custom_agg}
    )
    print(df, "\n")
    # If any key in any dictionnary is < 0 then the order is not possible (more sales than receptionss)
    df.loc[:, "sizes_list"] = df.sizes_list.map(validate_date)
    print(df, "\n")
    return "Possible" if df.sizes_list.all() else "Not Possible"


main_function(df)

   id date_added      type  sizes                sizes_list
0   1    2016-02  Received  7/8/9  {'7': 1, '8': 1, '9': 1}
1   1    2016-02      Sold    8/9          {'8': 1, '9': 1}
2   2    2016-03  Received  36/37        {'36': 1, '37': 1} 

   id date_added      type  sizes                sizes_list
0   1    2016-02  Received  7/8/9  {'7': 1, '8': 1, '9': 1}
1   1    2016-02      Sold    8/9        {'8': -1, '9': -1}
2   2    2016-03  Received  36/37        {'36': 1, '37': 1} 

   id date_added                sizes_list
0   1    2016-02  {'8': 0, '9': 0, '7': 1}
1   2    2016-03        {'36': 1, '37': 1} 

   id date_added sizes_list
0   1    2016-02       True
1   2    2016-03       True 



'Possible'

In [4]:
df = pd.DataFrame(
    {
        "id": [1, 1, 2],
        "date_added": ["2016-02", "2016-02", "2016-03"],
        "type": ["Received", "Sold", "Received"],
        "sizes": ["7/8/9", "8/9/10", "36/37"],
    }
)
df

,id,date_added,type,sizes
0,1,2016-02,Received,7/8/9
1,1,2016-02,Sold,8/9/10
2,2,2016-03,Received,36/37


In [5]:
main_function(df)

   id date_added      type   sizes                 sizes_list
0   1    2016-02  Received   7/8/9   {'7': 1, '8': 1, '9': 1}
1   1    2016-02      Sold  8/9/10  {'8': 1, '9': 1, '10': 1}
2   2    2016-03  Received   36/37         {'36': 1, '37': 1} 

   id date_added      type   sizes                    sizes_list
0   1    2016-02  Received   7/8/9      {'7': 1, '8': 1, '9': 1}
1   1    2016-02      Sold  8/9/10  {'8': -1, '9': -1, '10': -1}
2   2    2016-03  Received   36/37            {'36': 1, '37': 1} 

   id date_added                          sizes_list
0   1    2016-02  {'8': 0, '9': 0, '7': 1, '10': -1}
1   2    2016-03                  {'36': 1, '37': 1} 

   id date_added sizes_list
0   1    2016-02      False
1   2    2016-03       True 



'Not Possible'